In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from google.colab import drive
import json
from sklearn.metrics import accuracy_score

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
actual_tags = []
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/tags_original.txt', 'r') as file:
    actual_tags = json.load(file)

In [ ]:
removed = []
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/sentences_without_tags.txt', 'r') as file:
    removed = json.load(file)

In [ ]:
print(len(removed))

331364


In [ ]:
def extract_features(sentence, index):
  return {
      'word':sentence[index],
      'is_first':index==0,
      'is_last':index ==len(sentence)-1,
      'prefix-1':sentence[index][0],
      'prefix-2':sentence[index][:2],
      'prefix-3':sentence[index][:3],
      'prefix-3':sentence[index][:4],
      'suffix-1':sentence[index][-1],
      'suffix-2':sentence[index][-2:],
      'suffix-3':sentence[index][-3:],
      'suffix-3':sentence[index][-4:],
      'prev_word':'' if index == 0 else sentence[index-1],
      'next_word':'' if index == 1 else sentence[index+1],
      'has_hyphen': '-' in sentence[index],
      'is_numeric': sentence[index].isdigit()
  }

In [ ]:
def transform_to_dataset(sentences):
  X, y = [], []
  for sents in sentences:
    for index in range(len(sents)):
      X.append(extract_features(sents, index))
  return X, actual_tags[0: 22000]

In [ ]:
X_, y_ = transform_to_dataset(removed[0: 11000])
for i in range(0, 100):
  print(X_[i], " ----------> ", y_[i])

{'word': 'आग', 'is_first': True, 'is_last': False, 'prefix-1': 'आ', 'prefix-2': 'आग', 'prefix-3': 'आग', 'suffix-1': 'ग', 'suffix-2': 'आग', 'suffix-3': 'आग', 'prev_word': '', 'next_word': 'की', 'has_hyphen': False, 'is_numeric': False}  ---------->  nn
{'word': 'की', 'is_first': False, 'is_last': True, 'prefix-1': 'क', 'prefix-2': 'की', 'prefix-3': 'की', 'suffix-1': 'ी', 'suffix-2': 'की', 'suffix-3': 'की', 'prev_word': 'आग', 'next_word': '', 'has_hyphen': False, 'is_numeric': False}  ---------->  psp
{'word': 'लिए', 'is_first': True, 'is_last': False, 'prefix-1': 'ल', 'prefix-2': 'लि', 'prefix-3': 'लिए', 'suffix-1': 'ए', 'suffix-2': 'िए', 'suffix-3': 'लिए', 'prev_word': '', 'next_word': 'सांडिआ', 'has_hyphen': False, 'is_numeric': False}  ---------->  psp
{'word': 'सांडिआ', 'is_first': False, 'is_last': True, 'prefix-1': 'स', 'prefix-2': 'सा', 'prefix-3': 'सांड', 'suffix-1': 'आ', 'suffix-2': 'िआ', 'suffix-3': 'ंडिआ', 'prev_word': 'लिए', 'next_word': '', 'has_hyphen': False, 'is_numeric'

In [ ]:
print(len(X_), len(y_))

22000 22000


In [ ]:
print(y_[0: 10])
print(actual_tags[0: 10])

['nn', 'psp', 'psp', 'nnpc', 'psp', 'nnpc', 'vaux', 'vaux', 'nnp', 'psp']
['nn', 'psp', 'psp', 'nnpc', 'psp', 'nnpc', 'vaux', 'vaux', 'nnp', 'psp']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, y_, train_size=0.75)

In [ ]:
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X_)
X_train = dict_vectorizer.transform(X_train)
X_test = dict_vectorizer.transform(X_test)

In [ ]:
X_ = dict_vectorizer.transform(X_)

In [ ]:
model_ml = SVC()

In [ ]:
model_ml.fit(X_train, y_train)
print("Accuracy: {}".format(model_ml.score(X_test, y_test)))

Accuracy: 81.81818181


In [ ]:
final_predictions_ml = []
for i in X_test:
  final_predictions_ml.extend(model_ml.predict([i]))

In [ ]:
ml_tags = []
for i in X_:
  ml_tags.extend(model_ml.predict([i]))

In [ ]:
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/svm_predictions.txt', 'w+') as file:
    json.dump(ml_tags, file)